In [1]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten,Conv2D,Activation,Dropout
from keras import backend as K
import keras
from keras.models import Sequential, Model
from keras.models import load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import MaxPool2D
import tensorflow as tf

ModuleNotFoundError: No module named 'cv2'

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.set_visible_devices(physical_devices[0], 'GPU')

IndexError: list index out of range

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
for i, device in enumerate(physical_devices):
    print(i, device.name)

In [29]:
train_path="E:/SBIR/Selected/train"
test_path="E:/SBIR/Selected/test"
class_names=os.listdir(train_path)
class_names_test=os.listdir(test_path)

In [30]:
print(class_names)
print(class_names_test)

['airplane', 'apple', 'banana', 'bicycle', 'car', 'dog', 'door', 'ladder', 'moon', 'sheep', 'table', 'tree', 'wheel']
['airplane', 'apple', 'banana', 'bicycle', 'car', 'dog', 'door', 'ladder', 'moon', 'sheep', 'table', 'tree', 'wheel']


In [43]:
train_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)
test_datagen = ImageDataGenerator()

In [44]:
train_generator = train_datagen.flow_from_directory(train_path,target_size=(224, 224),batch_size=32,shuffle=True,class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path,target_size=(224,224),batch_size=32,shuffle=False,class_mode='categorical')

Found 780 images belonging to 13 classes.
Found 260 images belonging to 13 classes.


In [2]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="E:/SBIR/Selected/train",target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="E:/SBIR/Selected/test", target_size=(224,224))

Found 780 images belonging to 13 classes.
Found 260 images belonging to 13 classes.


In [45]:
def VGG16():
    model = Sequential()
    model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
    model.add(Flatten(name='flatten'))
    model.add(Dense(256, activation='relu', name='fc1'))
    model.add(Dense(128, activation='relu', name='fc2'))
    model.add(Dense(13, activation='sigmoid', name='output'))
    return model

In [46]:
model=VGG16()

In [47]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_40 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_42 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 56, 56, 128)      0         
 g2D)                                                 

In [48]:
from keras.models import Model
Vgg16 = Model(inputs=model.input, outputs=model.get_layer('vgg16').output)

In [49]:
Vgg16.load_weights("E:/SBIR/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [50]:
for layer in Vgg16.layers:
    layer.trainable = False

In [51]:
for layer in model.layers:
    print(layer, layer.trainable)

<keras.layers.convolutional.conv2d.Conv2D object at 0x000001E4953627C0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001E49F1EA1F0> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x000001E49F1F8250> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001E49F1EAA60> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001E49F23BD30> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x000001E49F1F8370> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001E49F1EA550> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001E49F22A190> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001E49F2344F0> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x000001E49F223670> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001E49F242430> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001E49F2420D0> False
<keras.layers.convolutional.conv2d.Conv2D o

In [52]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

e:\Ana\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [53]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_40 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_42 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 56, 56, 128)      0         
 g2D)                                                 

In [60]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
es = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
H = model.fit_generator(train_generator,validation_data=test_generator,epochs=100,verbose=1,callbacks=[es])

C:\Users\Yahya Akel\AppData\Local\Temp\ipykernel_140\1032787172.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(train_generator,validation_data=test_generator,epochs=100,verbose=1,callbacks=[es])


Epoch 1/100
25/25 [==============================] - 94s 4s/step - loss: 1.1668 - accuracy: 0.8179 - val_loss: 0.8857 - val_accuracy: 0.8500
Epoch 2/100
25/25 [==============================] - 83s 3s/step - loss: 0.7304 - accuracy: 0.8615 - val_loss: 1.1474 - val_accuracy: 0.8462
Epoch 3/100
13/25 [==============>...............] - ETA: 28s - loss: 1.2486 - accuracy: 0.8359

KeyboardInterrupt: 

In [ ]:
# Renders the charts for training accuracy and loss.
def render_training_history(training_history):
    loss = training_history.history['loss']
    val_loss = training_history.history['val_loss']

    accuracy = training_history.history['accuracy']
    val_accuracy = training_history.history['val_accuracy']

    plt.figure(figsize=(14, 4))

    plt.subplot(1, 2, 1)
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.plot(loss, label='Training set')
    plt.plot(val_loss, label='Test set', linestyle='--')
    plt.legend()
    plt.grid(linestyle='--', linewidth=1, alpha=0.5)

    plt.subplot(1, 2, 2)
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.plot(accuracy, label='Training set')
    plt.plot(val_accuracy, label='Test set', linestyle='--')
    plt.legend()
    plt.grid(linestyle='--', linewidth=1, alpha=0.5)

    plt.show()

In [ ]:
render_training_history(training_history)

In [ ]:
import pickle
with open('model_cnnVGG.pkl', 'wb') as file:
      pickle.dump(model, file)